In [31]:
import time, random
from tqdm import tqdm

import requests
from urllib import parse
from bs4 import BeautifulSoup

base_url = 'https://movie.naver.com/movie/point/af/list.naver?&page={}'

data_list = []

for page in tqdm(range(1,111)):
  url = base_url.format(page)
  res = requests.get(url)
  soup = BeautifulSoup(res.text)
  titles = soup.select('tr > td.title')
  for title in titles:
    movie = title.select_one('a.movie').text.strip()
    sentence = title.select_one('br').next_sibling.strip()
    if not sentence:
      continue
    score = title.select_one('div.list_netizen_score > em').text.strip()
    data_list.append((str(movie),str(sentence),int(score)))
  interval = round(random.uniform(0.2,1.1),2)
  time.sleep(interval) 

100%|██████████| 110/110 [02:41<00:00,  1.47s/it]


In [8]:
from os import write
import csv
fields = ['movie','sentence','score']

with open('samples.csv','w',encoding='utf-8-sig') as f:
  write = csv.writer(f, delimiter=',',
                     quotechar='"',
                     quoting = csv.QUOTE_MINIMAL)
  write.writerow(fields)
  write.writerows(data_list)

with open('samples.csv','r',encoding='utf-8-sig') as f:
  reader = csv.reader(f, delimiter=',', 
                      quotechar='"',
                      quoting = csv.QUOTE_MINIMAL)
  
  for row in reader:
    print(row)

['movie', 'sentence', 'score']
['더 킬러: 죽어도 되는 아이', '평점알바들이 많아서 고민하다 봤는데 액션 정말좋고 스토리 이상하지않고 킬링타임용으로 좋은 영화입니다.전 재밌게봤어용', '10']
['탑건: 매버릭', '상영 끝나기전에 꼭 보세요.', '10']
['보헤미안 랩소디', '가슴을 울리는 여왕의 연대기', '10']
['헤어질 결심', '몇가지 옥의 티가 있음.공대 출신인 이정현이 단순히 부재중 전화 2통 왔다는 정황증거만으로 이주임과 정분이 나는 장면.자부심이 중요하다는 박해일이 정작 탕웨이에게 마음을 뺏기는 것. 뭐 그럴 수도 있음. 자기가 잘 잘 수 있게 해주니깐. 근데 그럴거면 첫 만남씬에 탕웨이가 왜 그리 남루한 차림으로 했을까. 아주 매력적으로 보이지 않았음. 첫 눈에 뿅 간다는게 생각되지 않을 정도로...', '10']
['탑건: 매버릭', '영화관에서 안 보면 후회할 영화', '10']
['헤어질 결심', '그래서 말하고 싶은게 뭐고?', '8']
['탑건: 매버릭', '최고다 진짜 너뮤 재밌었다', '10']
['탑건: 매버릭', '지릴까봐 빤스2장 챙겨갔는데 옆사람꺼까지 빌렷읍니다.', '10']
['탑건: 매버릭', '미국..미국..미국...그리고 또 미국아...알면서 또 봤다', '4']
['마녀(魔女) Part2. The Other One', '다크한 분위기가 좋았고 여주가 너무 귀여웠어요', '8']
['마녀(魔女) Part2. The Other One', '다크한 분위기가 좋았고 여주가 너무 귀여웠어요', '8']
['헤어질 결심', '감동이 있거나 그런건 아니지만 여운이 깊게 남는것같아요', '10']
['마녀(魔女) Part2. The Other One', '어우 영화내내 욕밖에 없네.. 욕이 없음 대사를 못씁니까? ㅆx만 천번은 넘게 들은듯', '4']
['블러드 라인', '알몸으로 시엄마 마주치고 침대로 돌아갈때 대역이 너무 티가난다. 빤스가 없다가 생기고....참네 일 이따우로 할꺼냐 

In [25]:
class RawMovieReview:
  def __init__(self, fileName: str='samples.csv'):
    self._fileName = fileName
    self._data = []
    
    with open(self._fileName,'r',encoding = 'utf-8-sig') as f:
      reader = csv.reader(f, delimiter=',', 
                          quotechar='"',
                          quoting = csv.QUOTE_MINIMAL)
      for row in reader:
        self._data.append(row)
    self._data= self._data[1:]

  def __len__(self):
    return len(self._data)
  def __getitem__(self, index):
    self._title = str(self._data[index][0])
    self._sentence= str(self._data[index][1])
    self._score = int(self._data[index][2])
    return tuple([self._title, self._sentence, self._score])

In [26]:
r_data = RawMovieReview()
print(f"데이터 타입: {type(r_data[100])}")
print(f"한줄 평: {r_data[100]}")
print(f"데이터 한줄 평 개수 : {len(r_data)}")
print(type(r_data[100][0]))
print(type(r_data[100][1]))
print(type(r_data[100][2]))

데이터 타입: <class 'tuple'>
한줄 평: ('복수는 나의 것', '올드보이와 금자씨는 개봉때보고 복수3부작이라고해서 봤다가 끔찍했던 씬에 대한 기억이 너무 크게만 남았었는데 다시 보니 박찬욱감독 특유의 연출이 곳곳에 녹아있어 최근 작품들과의 연결고리를 찾는 재미로 감상을 했음. 피칠갑에 강렬한 메세지를 주고 있는와중에도 간간히 위트있는 장면이 숨어있고 20년전 배우들의 연기를 감상하는 맛도 있음. jsa같이 대중적 작품으로 빅히트를 치고 바로 나온 작품이라니. 감독의 자신감과 자기색깔에 대한 방향이 확실히 보이는 작품임.', 8)
데이터 한줄 평 개수 : 1019
<class 'str'>
<class 'str'>
<class 'int'>


In [41]:
class MovieReview(RawMovieReview):
    def __init__(self, fileName: str = 'samples.csv', ScoreThr: int = 5): # Score thr 1 ~ 10
        super().__init__()
        self._ScoreThr = ScoreThr
        
    def __getitem__(self, index):
        if int(super().__getitem__(index)[2]) >= self._ScoreThr:
            return str(super().__getitem__(index)[1]), True
        else:
            return str(super().__getitem__(index)[1]), False

In [42]:
review = MovieReview()
for i in range(1,100):
    print(review[i])

('상영 끝나기전에 꼭 보세요.', True)
('가슴을 울리는 여왕의 연대기', True)
('몇가지 옥의 티가 있음.공대 출신인 이정현이 단순히 부재중 전화 2통 왔다는 정황증거만으로 이주임과 정분이 나는 장면.자부심이 중요하다는 박해일이 정작 탕웨이에게 마음을 뺏기는 것. 뭐 그럴 수도 있음. 자기가 잘 잘 수 있게 해주니깐. 근데 그럴거면 첫 만남씬에 탕웨이가 왜 그리 남루한 차림으로 했을까. 아주 매력적으로 보이지 않았음. 첫 눈에 뿅 간다는게 생각되지 않을 정도로...', True)
('영화관에서 안 보면 후회할 영화', True)
('그래서 말하고 싶은게 뭐고?', True)
('최고다 진짜 너뮤 재밌었다', True)
('지릴까봐 빤스2장 챙겨갔는데 옆사람꺼까지 빌렷읍니다.', True)
('미국..미국..미국...그리고 또 미국아...알면서 또 봤다', False)
('다크한 분위기가 좋았고 여주가 너무 귀여웠어요', True)
('다크한 분위기가 좋았고 여주가 너무 귀여웠어요', True)
('감동이 있거나 그런건 아니지만 여운이 깊게 남는것같아요', True)
('어우 영화내내 욕밖에 없네.. 욕이 없음 대사를 못씁니까? ㅆx만 천번은 넘게 들은듯', False)
('알몸으로 시엄마 마주치고 침대로 돌아갈때 대역이 너무 티가난다. 빤스가 없다가 생기고....참네 일 이따우로 할꺼냐 귀네쓰??', False)
('흥미진진하고 재밌었어요!!', True)
('대실망나의 토르를 다 망가뜨려놨네.상남자토르를 완전 코메디언 만듬.웃기지도 않고, 억지 3류 코미디영화저런 명배우들 쓰다가 영화를 저렇게 밖에 못만드는지 ...', False)
('방금 4회차 보고옴. 담주 마지막 5회차로 나도 헤어질결심했다. ㅠ해준이 아직도 서래 찾고있겠네', True)
('배우들의 연기..', True)
('톰크루즈와 동시대를 살아감에 감사하다', True)
('인물들 간에 계속 줄타기를 하는 듯한 아슬아슬한 분위기가 인상적이었어요', True)
('첫화면 항공모함 전